# Predicting Depression from Routine Survey Data using Keras

# Getting Started

In [1]:
import pandas as pd
import numpy as np
import re
from keras.models import Sequential
from keras.layers import Dense
from keras.preprocessing import sequence

In [2]:
#train set
df_train = pd.read_csv('D:/MH/train.csv')
# test set
df_test = pd.read_csv('D:/MH//test.csv')

#show the shape of the train dataframe
df_train.shape

(1143, 75)

In [45]:
df_train

,surveyid,village,survey_date,femaleres,age,married,children,hhsize,edu,hh_children,...,given_mpesa,amount_given_mpesa,received_mpesa,amount_received_mpesa,net_mpesa,saved_mpesa,amount_saved_mpesa,early_survey,depressed,day_of_week
0,926,91,136,1,28.0,1,4,6,10,0,...,0,0.0,0,0.000000,0.000000,1,0.000000,0,0,5
1,747,57,142,1,23.0,1,3,5,8,0,...,0,0.0,1,4.804611,4.804611,0,0.000000,0,1,3
2,1190,115,24,1,22.0,1,3,5,9,0,...,0,0.0,0,8.007685,8.007685,1,0.000000,0,0,5
3,1065,97,139,1,27.0,1,2,4,10,2,...,0,0.0,0,0.000000,0.000000,1,1.249199,0,0,0
4,806,42,74,0,59.0,0,4,6,10,4,...,0,0.0,0,0.000000,0.000000,0,0.000000,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1138,927,152,155,1,36.0,0,4,5,9,0,...,0,0.0,0,0.000000,0.000000,0,0.000000,0,1,4
1139,1039,104,79,1,23.0,1,0,2,10,0,...,0,0.0,0,0.000000,0.000000,1,0.000000,0,0,4
1140,428,54,136,1,28.0,1,5,7,10,5,...,0,0.0,0,0.000000,0.000000,0,0.000000,0,0,5
1141,23,3,24,1,33.0,0,4,5,9,0,...,0,0.0,0,0.000000,0.000000,0,0.000000,0,0,5


In [46]:
df_test 

,surveyid,village,survey_date,femaleres,age,married,children,hhsize,edu,hh_children,...,given_mpesa,amount_given_mpesa,received_mpesa,amount_received_mpesa,net_mpesa,saved_mpesa,amount_saved_mpesa,early_survey,depressed,day_of_week
0,901,181,14-Nov-61,1,21,0,3,4,10,3,...,0,0.0,0,0.000000,0.000000,1,0.000000,0,0.0,3
1,498,47,18-Sep-61,1,44,1,6,8,6,6,...,0,0.0,0,0.000000,0.000000,0,0.000000,0,0.0,2
2,710,8,03-Sep-61,1,23,1,1,3,7,1,...,0,0.0,0,0.000000,0.000000,1,0.080077,0,0.0,1
3,433,25,01-Nov-60,1,67,0,0,1,1,0,...,0,0.0,0,0.000000,0.000000,0,0.000000,1,1.0,3
4,44,2,18-May-60,1,40,1,4,6,7,4,...,0,0.0,0,0.000000,0.000000,0,0.000000,1,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,255,22,24-Sep-61,1,25,1,1,3,7,0,...,0,0.0,0,0.000000,0.000000,0,0.000000,0,NaN,1
282,547,69,23-Nov-61,1,28,1,4,6,10,4,...,0,0.0,0,0.000000,0.000000,1,0.000000,0,NaN,5
283,893,184,14-Nov-61,1,66,0,0,1,1,0,...,0,0.0,0,0.000000,0.000000,0,0.000000,0,NaN,3
284,363,75,28-Nov-61,1,51,1,1,3,12,0,...,0,0.0,0,0.000000,0.000000,0,0.000000,0,NaN,3


# Data Cleaning and Processing
 The function below computes and prints the percentage of missing values for each column.

In [3]:
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("The dataset has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [4]:
# Get the columns with > 50% missing
missing_df = missing_values_table(df_train);
missing_columns = list(missing_df[missing_df['% of Total Values'] > 50].index)
print('\n','%d columns will be deleted.' % len(missing_columns))

# Drop the columns with 50% missing data
df_train = df_train.drop(columns = list(missing_columns))

The dataset has 75 columns.
There are 23 columns that have missing values.

 4 columns will be deleted.


In [17]:
import sys
sys.stdin.encoding 
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

In [11]:
# Create a label (category) encoder object
encoder = preprocessing.LabelEncoder()

# fitting the encoder to the "survey_date" column
encoder.fit(df_train['survey_date'])

# Apply the fitted encoder to the "survey_date" to transform categories into integers
encoded_train = encoder.transform(df_train['survey_date'])
# encoded_test = encoder.transform(df_test['survey_date'])

#assign the tranformed column back to the dataframe
df_train['survey_date'] = encoded_train

ow that we’ve processed our data to a considerable level, it’s time to specify the input and target variables. Our input will be every column except the depressed column, since that’s what we’re attempting to predict—hence, it’s our target variable.
We also handle any missing values in the columns with less that 50% null values using SKlearn’s Imputer class. The data is then split into train and test sets, and a random seed of 5 is specified for the purpose of reproducing the results.

In [18]:
# split data into train and test sets
X = df_train.drop(["depressed"], axis=1)

# fill missing values with mean column values
imputer = SimpleImputer()
transformed_X = imputer.fit_transform(X)

y = df_train.depressed

seed = 5
test_size = 0.33

X_train, X_test, y_train, y_test = train_test_split(transformed_X, y, test_size=test_size, random_state=seed)

# Creating the Classification Model
We build a simple neural network classification model using the Keras library. Keras is a user-friendly neural network library written in Python that defines models as a sequence of layers.
Using the add function, we add two input layers and an output layer, all of the type Dense, meaning that the layers are fully connected. Each input layer has 36 neurons, and the first layer expects 70 inputs while the output layer has 1 neuron to predict the outcome. We use the sigmoid function on the output layer to ensure our network output is between 0 and 1, while the relu function is used in the input layers.


In [41]:
# create model
model = Sequential()
model.add(Dense(36, input_dim=70, activation='relu'))
model.add(Dense(36, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the Model
Now that the model is defined, we can compile it. In our case, TensorFlow backend is used for compiling, and it involves automatically choosing the best way to represent the network for training and making predictions. Training a network simply means finding the best set of weights to make predictions for this problem.
We therefore specify the loss function to be used to evaluate a set of weights, the optimizer used to search through different weights for the network, and metrics to evaluate the model’s performance. For a binary classification problem like this one, we use binary_crossentropy for the logarithmic loss, adam for gradient descent, and accuracy for the metric parameter.


In [43]:
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Evaluate the Model
We finally evaluate the model for our test data.

In [44]:
scores = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

12/12 [==============================] - 0s 402us/step - loss: 14.5219 - accuracy: 0.8715

accuracy: 86.24%


In [ ]:
The model achieved accuracy of   86.24% This accuracy is Slighly higher than  that of the random forest baseline model below, whose accuracy evaluated to 85.71%.

In [48]:

prediction =model.predict(X_test)

In [49]:
print(prediction)

[[5.05604780e-10]
 [6.80511676e-38]
 [2.46849330e-27]
 [7.33792252e-18]
 [4.10102226e-29]
 [1.86493786e-27]
 [1.17610966e-38]
 [6.38306809e-38]
 [1.13623905e-30]
 [3.28723723e-24]
 [6.61562484e-16]
 [0.00000000e+00]
 [0.00000000e+00]
 [5.01381372e-32]
 [2.10294608e-08]
 [7.35899871e-34]
 [0.00000000e+00]
 [1.37093327e-28]
 [1.46091011e-21]
 [0.00000000e+00]
 [1.09328600e-37]
 [4.82883149e-29]
 [0.00000000e+00]
 [0.00000000e+00]
 [5.20201801e-25]
 [0.00000000e+00]
 [0.00000000e+00]
 [2.51566828e-23]
 [1.33675915e-09]
 [1.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [5.86397285e-34]
 [0.00000000e+00]
 [0.00000000e+00]
 [1.54336928e-37]
 [0.00000000e+00]
 [6.84463555e-16]
 [1.76007284e-19]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [7.40962011e-26]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [2.11685468e-30]
 [0.00000000e+00]
 [0.00000000e+00]
 [1.53152891e-36]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.000000

# Baseline Model

In [33]:
#Baseline Model
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
#logreg = LogisticRegression()
logreg = RandomForestClassifier()
logreg.fit(X_train, y_train)
accuracy = logreg.score(X_test, y_test)

y_pred = logreg.predict(X_test)
accuracy = metrics.accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 85.98%


In [30]:
pip install XGBOOST

In [34]:
import xgboost as xgb

In [37]:
logreg  = xgb.XGBClassifier()
logreg.fit(X_train, y_train)
# accuracy = logreg.score(X_test, y_test)

y_pred = logreg.predict(X_test)
accuracy = metrics.accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

C:\Users\Admin\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:07:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 84.39%
